In [ ]:
# !pip install pycaret

In [ ]:
# !pip install pycaret[full]

## Load data and preprocessing

In [1]:
import statistics as st
%matplotlib inline  
import matplotlib.dates as mdates
import pandas as pd
from pandas import DataFrame
import datetime
import numpy as np

from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Optuna Libraries
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

# LGBM Regressor
import lightgbm as lgb
from lightgbm import LGBMRegressor

### Load

#### Greenhouse Climate
- 데이터 구성
  - 5분 간격으로 측정됨
  - Tair : Greenhouse Air Temperature
  - Rhair : Greenhouse relative humidity
  - CO2air : CO2 greenhosue 
  - co2_dos : CO2 dosing
  - Tot_PAR : total inside PAR(Sun + HPS + LED)

In [3]:
import pandas as pd

gc = pd.read_csv('/home/jy/TimeSeries_Analysis/AGIC/GreenhouseClimate_automato_modified.csv')
gc.head()

,time,AssimLight,BlackScr,CO2air,Cum_irr,EC_drain_PC,EnScr,HumDef,PipeGrow,PipeLow,...,t_rail_min_sp,t_rail_min_vip,t_vent_sp,t_ventlee_vip,t_ventwind_vip,water_sup,water_sup_intervals_sp_min,water_sup_intervals_vip_min,window_pos_lee_sp,window_pos_lee_vip
0,2019-12-16 0:00,0.0,35.000000,468.000000,0.26,1.78,96.0,7.32,34.5,50.4,...,NaN,0.0,NaN,25.0,26.0,2.0,NaN,1000.0,NaN,1.2
1,2019-12-16 0:05,0.0,85.000001,508.999999,0.52,1.78,96.0,7.08,33.2,48.7,...,NaN,0.0,NaN,25.0,26.0,4.0,NaN,1000.0,NaN,1.2
2,2019-12-16 0:10,0.0,95.999999,498.000001,0.52,1.78,96.0,7.48,32.4,47.1,...,NaN,0.0,NaN,25.0,26.0,4.0,NaN,1000.0,NaN,1.2
3,2019-12-16 0:15,0.0,96.000000,491.000001,0.52,1.76,96.0,6.55,31.7,45.4,...,NaN,0.0,NaN,25.0,26.0,4.0,NaN,1000.0,NaN,1.2
4,2019-12-16 0:20,0.0,95.400000,514.999999,0.52,1.78,95.4,6.71,30.9,44.2,...,NaN,0.0,NaN,25.0,26.0,4.0,NaN,1000.0,NaN,1.2


In [4]:
gc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47809 entries, 0 to 47808
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   time                         47809 non-null  object 
 1   AssimLight                   47738 non-null  float64
 2   BlackScr                     47738 non-null  float64
 3   CO2air                       47738 non-null  float64
 4   Cum_irr                      47738 non-null  float64
 5   EC_drain_PC                  47738 non-null  float64
 6   EnScr                        47738 non-null  float64
 7   HumDef                       47738 non-null  float64
 8   PipeGrow                     47738 non-null  float64
 9   PipeLow                      47738 non-null  float64
 10  Rhair                        47738 non-null  float64
 11  Tair                         47738 non-null  float64
 12  Tot_PAR                      47738 non-null  float64
 13  Tot_PAR_Lamps   

In [5]:
gc.describe()

,AssimLight,BlackScr,CO2air,Cum_irr,EC_drain_PC,EnScr,HumDef,PipeGrow,PipeLow,Rhair,...,t_rail_min_sp,t_rail_min_vip,t_vent_sp,t_ventlee_vip,t_ventwind_vip,water_sup,water_sup_intervals_sp_min,water_sup_intervals_vip_min,window_pos_lee_sp,window_pos_lee_vip
count,47738.000000,47738.000000,47738.000000,47738.000000,47738.000000,47738.000000,47738.000000,47738.000000,47738.000000,47738.000000,...,47022.000000,47738.000000,35233.000000,47738.000000,47738.000000,47738.000000,46399.000000,47738.000000,47094.000000,47738.000000
mean,52.953622,31.658652,692.278834,2.791302,5.162962,47.816932,3.414497,22.962949,9.319944,84.928065,...,6.404173,6.282532,31.056787,30.657529,30.662755,21.471553,762.612548,776.215447,6.580428,6.533770
std,49.913208,44.069875,149.081948,2.534420,1.470146,44.892454,2.623333,16.636085,16.726319,8.147000,...,10.635707,10.567635,2.648992,2.649443,2.636046,19.495540,619.633320,627.017115,12.307339,12.291332
min,0.000000,0.000000,343.000001,0.000000,0.110000,0.000000,0.620000,0.000000,0.000000,33.400000,...,0.000000,0.000000,13.000000,13.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,591.000000,0.520000,4.600000,0.000000,1.790000,0.000000,0.000000,82.700000,...,0.000000,0.000000,30.000000,30.000000,30.000000,4.000000,55.000000,60.000000,0.000000,0.000000
50%,100.000000,0.000000,678.000000,2.080000,5.080000,64.999996,2.590000,31.900000,0.000000,87.100000,...,0.000000,0.000000,32.000000,30.000000,30.000000,16.000000,1080.000000,1080.000000,1.000000,1.000000
75%,100.000000,91.300000,762.999997,4.420000,5.770000,94.600000,4.020000,34.700000,0.000005,90.300000,...,15.000000,15.000000,32.000000,32.000000,32.000000,34.000000,1080.000000,1080.000000,8.000000,8.000000
max,100.000000,100.000000,1959.000000,9.360000,10.409999,100.000000,25.340000,61.600000,63.800000,96.500000,...,60.000000,60.000000,36.000000,36.000000,36.000000,72.000000,1950.000000,2000.000000,100.000000,100.000000


In [6]:
gc.isnull().sum()

time                               0
AssimLight                        71
BlackScr                          71
CO2air                            71
Cum_irr                           71
EC_drain_PC                       71
EnScr                             71
HumDef                            71
PipeGrow                          71
PipeLow                           71
Rhair                             71
Tair                              71
Tot_PAR                           71
Tot_PAR_Lamps                      0
VentLee                           71
Ventwind                          71
assim_sp                         785
assim_vip                         71
co2_dos                            2
co2_sp                           754
co2_vip                           71
dx_sp                            718
dx_vip                            71
int_blue_sp                      173
int_blue_vip                   22152
int_farred_sp                     14
int_farred_vip                 22152
i

In [7]:
gc_ = gc[['time','Tair', 'Rhair', 'CO2air', 'co2_dos', 'Tot_PAR']]
gc_.head()

,time,Tair,Rhair,CO2air,co2_dos,Tot_PAR
0,2019-12-16 0:00,21.3,60.7,468.000000,NaN,0.0
1,2019-12-16 0:05,21.5,62.4,508.999999,0.001405,0.0
2,2019-12-16 0:10,22.0,61.4,498.000001,0.001311,0.0
3,2019-12-16 0:15,21.3,64.8,491.000001,0.000017,0.0
4,2019-12-16 0:20,21.6,64.6,514.999999,0.000019,0.0


In [8]:
import datetime
gc_['time'] = pd.to_datetime(gc_['time'])

gc_.dtypes

/tmp/ipykernel_39869/780101806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gc_['time'] = pd.to_datetime(gc_['time'])


time       datetime64[ns]
Tair              float64
Rhair             float64
CO2air            float64
co2_dos           float64
Tot_PAR           float64
dtype: object

In [9]:
gc2 = gc_.fillna(method = 'ffill')
gc2.isnull().sum()

time       0
Tair       0
Rhair      0
CO2air     0
co2_dos    1
Tot_PAR    0
dtype: int64

In [10]:
gc2_ = gc2.fillna(method = 'backfill')
gc2_.isnull().sum()

time       0
Tair       0
Rhair      0
CO2air     0
co2_dos    0
Tot_PAR    0
dtype: int64

In [11]:
gc2_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47809 entries, 0 to 47808
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   time     47809 non-null  datetime64[ns]
 1   Tair     47809 non-null  float64       
 2   Rhair    47809 non-null  float64       
 3   CO2air   47809 non-null  float64       
 4   co2_dos  47809 non-null  float64       
 5   Tot_PAR  47809 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 2.2 MB


In [12]:
gc2_.head()

,time,Tair,Rhair,CO2air,co2_dos,Tot_PAR
0,2019-12-16 00:00:00,21.3,60.7,468.000000,0.001405,0.0
1,2019-12-16 00:05:00,21.5,62.4,508.999999,0.001405,0.0
2,2019-12-16 00:10:00,22.0,61.4,498.000001,0.001311,0.0
3,2019-12-16 00:15:00,21.3,64.8,491.000001,0.000017,0.0
4,2019-12-16 00:20:00,21.6,64.6,514.999999,0.000019,0.0


In [13]:
#gc3 = gc2_.resample(rule = 'H').mean()

#### Weather data
- T_out : outside temperature
- TH_out : outside relative humidity
- I_glob : solar radiation
- Windsp : wind speed

In [14]:
w = pd.read_csv('/home/jy/TimeSeries_Analysis/AGIC/Weather_modified.csv')
w.head()

,time,T_out,RH_out,P_out,I_glob,u_wind,T_sky,PARout,Pyrgeo,RadSum,Winddir,Windsp
0,2019-12-16 0:00,6.9,81.0,NaN,0.0,4.7,NaN,0.0,-72.0,215.0,32.0,4.7
1,2019-12-16 0:05,6.9,81.0,NaN,0.0,4.7,NaN,0.0,-73.0,0.0,32.0,4.7
2,2019-12-16 0:10,6.9,80.0,NaN,0.0,4.7,NaN,0.0,-76.0,0.0,32.0,4.7
3,2019-12-16 0:15,6.9,80.0,NaN,0.0,4.7,NaN,0.0,-77.0,0.0,32.0,4.7
4,2019-12-16 0:20,6.7,81.0,NaN,0.0,4.7,NaN,0.0,-75.0,0.0,32.0,4.7


In [15]:
w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47809 entries, 0 to 47808
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   time     47809 non-null  object 
 1   T_out    47738 non-null  float64
 2   RH_out   47738 non-null  float64
 3   P_out    0 non-null      float64
 4   I_glob   47738 non-null  float64
 5   u_wind   47738 non-null  float64
 6   T_sky    0 non-null      float64
 7   PARout   47738 non-null  float64
 8   Pyrgeo   47738 non-null  float64
 9   RadSum   47738 non-null  float64
 10  Winddir  47738 non-null  float64
 11  Windsp   47738 non-null  float64
dtypes: float64(11), object(1)
memory usage: 4.4+ MB


In [16]:
w.describe()

,T_out,RH_out,P_out,I_glob,u_wind,T_sky,PARout,Pyrgeo,RadSum,Winddir,Windsp
count,47738.000000,47738.000000,0.0,47738.000000,47738.000000,0.0,47738.000000,47738.000000,47738.000000,47738.000000,47738.000000
mean,9.025342,74.873539,NaN,125.296703,4.982486,NaN,250.270026,-52.871381,477.410229,28.636369,4.982486
std,4.229593,19.899552,NaN,222.006296,3.849020,NaN,443.856662,36.340826,724.258902,26.860777,3.849020
min,-0.200000,10.000000,NaN,0.000000,0.000000,NaN,0.000000,-136.000000,0.000000,1.000000,0.000000
25%,6.300000,62.000000,NaN,0.000000,2.200000,NaN,0.000000,-84.000000,0.000000,4.000000,2.200000
50%,8.400000,79.000000,NaN,0.000000,4.200000,NaN,1.000000,-54.000000,122.000000,32.000000,4.200000
75%,11.000000,91.000000,NaN,146.000000,7.100000,NaN,287.000000,-17.000000,613.000000,32.000004,7.100000
max,27.300000,100.000000,NaN,1505.000000,21.900000,NaN,2010.000000,0.000000,2999.000000,128.000000,21.900000


In [17]:
w.columns

Index(['time', 'T_out', 'RH_out', 'P_out', 'I_glob', 'u_wind', 'T_sky',
       'PARout', 'Pyrgeo', 'RadSum', 'Winddir', 'Windsp'],
      dtype='object')

In [18]:
w_ = w[['time', 'T_out','RH_out', 'I_glob', 'Windsp']]
w_.head()

,time,T_out,RH_out,I_glob,Windsp
0,2019-12-16 0:00,6.9,81.0,0.0,4.7
1,2019-12-16 0:05,6.9,81.0,0.0,4.7
2,2019-12-16 0:10,6.9,80.0,0.0,4.7
3,2019-12-16 0:15,6.9,80.0,0.0,4.7
4,2019-12-16 0:20,6.7,81.0,0.0,4.7


In [19]:
w_.dtypes

time       object
T_out     float64
RH_out    float64
I_glob    float64
Windsp    float64
dtype: object

In [20]:
w_['time'] = pd.to_datetime(w_['time'])

w_.dtypes

/tmp/ipykernel_39869/1084852719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w_['time'] = pd.to_datetime(w_['time'])


time      datetime64[ns]
T_out            float64
RH_out           float64
I_glob           float64
Windsp           float64
dtype: object

In [21]:
w_.isnull().sum()

time       0
T_out     71
RH_out    71
I_glob    71
Windsp    71
dtype: int64

In [22]:
w2 = w_.fillna(method = 'ffill')
w2.isnull().sum()

time      0
T_out     0
RH_out    0
I_glob    0
Windsp    0
dtype: int64

In [23]:
w2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47809 entries, 0 to 47808
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    47809 non-null  datetime64[ns]
 1   T_out   47809 non-null  float64       
 2   RH_out  47809 non-null  float64       
 3   I_glob  47809 non-null  float64       
 4   Windsp  47809 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 1.8 MB


#### merge

In [24]:
mg = pd.merge(gc2_, w2, on = 'time')
mg.head()

,time,Tair,Rhair,CO2air,co2_dos,Tot_PAR,T_out,RH_out,I_glob,Windsp
0,2019-12-16 00:00:00,21.3,60.7,468.000000,0.001405,0.0,6.9,81.0,0.0,4.7
1,2019-12-16 00:05:00,21.5,62.4,508.999999,0.001405,0.0,6.9,81.0,0.0,4.7
2,2019-12-16 00:10:00,22.0,61.4,498.000001,0.001311,0.0,6.9,80.0,0.0,4.7
3,2019-12-16 00:15:00,21.3,64.8,491.000001,0.000017,0.0,6.9,80.0,0.0,4.7
4,2019-12-16 00:20:00,21.6,64.6,514.999999,0.000019,0.0,6.7,81.0,0.0,4.7


In [25]:
mg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47809 entries, 0 to 47808
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   time     47809 non-null  datetime64[ns]
 1   Tair     47809 non-null  float64       
 2   Rhair    47809 non-null  float64       
 3   CO2air   47809 non-null  float64       
 4   co2_dos  47809 non-null  float64       
 5   Tot_PAR  47809 non-null  float64       
 6   T_out    47809 non-null  float64       
 7   RH_out   47809 non-null  float64       
 8   I_glob   47809 non-null  float64       
 9   Windsp   47809 non-null  float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 4.0 MB


In [26]:
mg = mg.set_index('time')

#### Crop Parameters
- 1주 간격으로 측정
  - stem_elong : stem growth
  - stem_thick : stem thickness

In [27]:
cp = pd.read_csv('/home/jy/TimeSeries_Analysis/AGIC/CropParameters_automato_modified__.csv')
cp.head()

,Time,Stem_elong,Stem_thick,Cum_trusses,stem_dens,plant_dens
0,2019-12-23 23:00,14.5,8.1,NaN,3.9,1.95
1,2019-12-30 23:00,29.5,10.8,1.3,3.9,1.95
2,2020-01-07 23:00,37.4,12.8,3.0,3.9,1.95
3,2020-01-14 23:00,34.0,12.5,4.7,3.9,1.95
4,2020-01-21 23:00,34.7,10.9,5.9,3.9,1.95


In [28]:
cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Time         23 non-null     object 
 1   Stem_elong   18 non-null     float64
 2   Stem_thick   18 non-null     float64
 3   Cum_trusses  18 non-null     float64
 4   stem_dens    23 non-null     float64
 5   plant_dens   23 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.2+ KB


In [29]:
cp['Time'] = pd.to_datetime(cp['Time'])

cp.dtypes

Time           datetime64[ns]
Stem_elong            float64
Stem_thick            float64
Cum_trusses           float64
stem_dens             float64
plant_dens            float64
dtype: object

In [30]:
cp_ = cp[['Time','Stem_elong', 'Stem_thick']]
cp_.head()

,Time,Stem_elong,Stem_thick
0,2019-12-23 23:00:00,14.5,8.1
1,2019-12-30 23:00:00,29.5,10.8
2,2020-01-07 23:00:00,37.4,12.8
3,2020-01-14 23:00:00,34.0,12.5
4,2020-01-21 23:00:00,34.7,10.9


In [31]:
cp_.isnull().sum()

Time          0
Stem_elong    5
Stem_thick    5
dtype: int64

In [32]:
cp_.head(25)

,Time,Stem_elong,Stem_thick
0,2019-12-23 23:00:00,14.5,8.1
1,2019-12-30 23:00:00,29.5,10.8
2,2020-01-07 23:00:00,37.4,12.8
3,2020-01-14 23:00:00,34.0,12.5
4,2020-01-21 23:00:00,34.7,10.9
5,2020-01-28 23:00:00,36.2,11.4
6,2020-02-04 23:00:00,32.4,10.6
7,2020-02-11 23:00:00,37.2,11.1
8,2020-02-18 23:00:00,37.8,11.3
9,2020-02-25 23:00:00,35.8,11.6


In [33]:
cp2 = cp_.dropna(how = 'any')
cp2.isnull().sum()

Time          0
Stem_elong    0
Stem_thick    0
dtype: int64

In [34]:
cp3 = cp2.rename(columns = {'Time':'time'})
cp3.head()

,time,Stem_elong,Stem_thick
0,2019-12-23 23:00:00,14.5,8.1
1,2019-12-30 23:00:00,29.5,10.8
2,2020-01-07 23:00:00,37.4,12.8
3,2020-01-14 23:00:00,34.0,12.5
4,2020-01-21 23:00:00,34.7,10.9


In [35]:
data = pd.merge(mg, cp3, on = 'time', how = 'left')

data.head()

,time,Tair,Rhair,CO2air,co2_dos,Tot_PAR,T_out,RH_out,I_glob,Windsp,Stem_elong,Stem_thick
0,2019-12-16 00:00:00,21.3,60.7,468.000000,0.001405,0.0,6.9,81.0,0.0,4.7,NaN,NaN
1,2019-12-16 00:05:00,21.5,62.4,508.999999,0.001405,0.0,6.9,81.0,0.0,4.7,NaN,NaN
2,2019-12-16 00:10:00,22.0,61.4,498.000001,0.001311,0.0,6.9,80.0,0.0,4.7,NaN,NaN
3,2019-12-16 00:15:00,21.3,64.8,491.000001,0.000017,0.0,6.9,80.0,0.0,4.7,NaN,NaN
4,2019-12-16 00:20:00,21.6,64.6,514.999999,0.000019,0.0,6.7,81.0,0.0,4.7,NaN,NaN


In [36]:
data.set_index('time', inplace = True)

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 47809 entries, 2019-12-16 00:00:00 to 2020-05-30 00:00:00
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Tair        47809 non-null  float64
 1   Rhair       47809 non-null  float64
 2   CO2air      47809 non-null  float64
 3   co2_dos     47809 non-null  float64
 4   Tot_PAR     47809 non-null  float64
 5   T_out       47809 non-null  float64
 6   RH_out      47809 non-null  float64
 7   I_glob      47809 non-null  float64
 8   Windsp      47809 non-null  float64
 9   Stem_elong  18 non-null     float64
 10  Stem_thick  18 non-null     float64
dtypes: float64(11)
memory usage: 4.4 MB


In [38]:
data.loc['2019-12-23 22:55:00']

Tair           17.2
Rhair          80.7
CO2air        540.0
co2_dos         0.0
Tot_PAR         0.0
T_out           7.6
RH_out         87.0
I_glob          0.0
Windsp          4.9
Stem_elong      NaN
Stem_thick      NaN
Name: 2019-12-23 22:55:00, dtype: float64

In [39]:
data.loc['2019-12-23 23:00:00']

Tair           17.100000
Rhair          80.700000
CO2air        552.999999
co2_dos         0.000000
Tot_PAR         0.000000
T_out           7.600000
RH_out         88.000000
I_glob          0.000000
Windsp          4.900000
Stem_elong     14.500000
Stem_thick      8.100000
Name: 2019-12-23 23:00:00, dtype: float64

In [40]:
data1 = data.loc[:'2019-12-23 22:55:00']
data1.head()

,Tair,Rhair,CO2air,co2_dos,Tot_PAR,T_out,RH_out,I_glob,Windsp,Stem_elong,Stem_thick
time,,,,,,,,,,,
2019-12-16 00:00:00,21.3,60.7,468.000000,0.001405,0.0,6.9,81.0,0.0,4.7,NaN,NaN
2019-12-16 00:05:00,21.5,62.4,508.999999,0.001405,0.0,6.9,81.0,0.0,4.7,NaN,NaN
2019-12-16 00:10:00,22.0,61.4,498.000001,0.001311,0.0,6.9,80.0,0.0,4.7,NaN,NaN
2019-12-16 00:15:00,21.3,64.8,491.000001,0.000017,0.0,6.9,80.0,0.0,4.7,NaN,NaN
2019-12-16 00:20:00,21.6,64.6,514.999999,0.000019,0.0,6.7,81.0,0.0,4.7,NaN,NaN


In [41]:
data1.tail()

,Tair,Rhair,CO2air,co2_dos,Tot_PAR,T_out,RH_out,I_glob,Windsp,Stem_elong,Stem_thick
time,,,,,,,,,,,
2019-12-23 22:35:00,17.2,80.7,537.000001,0.0,0.0,7.9,86.0,0.0,4.9,NaN,NaN
2019-12-23 22:40:00,17.2,81.0,549.999998,0.0,0.0,8.0,86.0,0.0,4.9,NaN,NaN
2019-12-23 22:45:00,17.3,81.2,558.999999,0.0,0.0,7.9,86.0,0.0,4.9,NaN,NaN
2019-12-23 22:50:00,17.2,80.9,549.500000,0.0,0.0,7.8,86.0,0.0,4.9,NaN,NaN
2019-12-23 22:55:00,17.2,80.7,540.000000,0.0,0.0,7.6,87.0,0.0,4.9,NaN,NaN


In [42]:
data2 = data.loc['2019-12-23 22:55:00':]
data2.head()

,Tair,Rhair,CO2air,co2_dos,Tot_PAR,T_out,RH_out,I_glob,Windsp,Stem_elong,Stem_thick
time,,,,,,,,,,,
2019-12-23 22:55:00,17.2,80.7,540.000000,0.0,0.0,7.6,87.0,0.0,4.9,NaN,NaN
2019-12-23 23:00:00,17.1,80.7,552.999999,0.0,0.0,7.6,88.0,0.0,4.9,14.5,8.1
2019-12-23 23:05:00,17.3,80.4,540.000002,0.0,0.0,7.6,89.0,0.0,4.9,NaN,NaN
2019-12-23 23:10:00,17.1,80.2,540.000000,0.0,0.0,7.4,89.0,0.0,4.9,NaN,NaN
2019-12-23 23:15:00,17.3,80.0,547.000000,0.0,0.0,7.4,90.0,0.0,4.9,NaN,NaN


In [43]:
data2.tail()

,Tair,Rhair,CO2air,co2_dos,Tot_PAR,T_out,RH_out,I_glob,Windsp,Stem_elong,Stem_thick
time,,,,,,,,,,,
2020-05-29 23:40:00,18.3,77.8,493.0,0.000122,4.999999e-01,15.1,71.0,0.0,4.3,NaN,NaN
2020-05-29 23:45:00,18.3,78.5,493.0,0.000121,5.000000e-01,15.0,71.0,0.0,4.3,NaN,NaN
2020-05-29 23:50:00,18.3,79.1,485.0,0.000121,1.680000e-08,15.0,71.0,0.0,3.8,NaN,NaN
2020-05-29 23:55:00,18.0,79.9,502.0,0.000120,0.000000e+00,15.0,71.0,0.0,3.8,NaN,NaN
2020-05-30 00:00:00,18.0,79.9,502.0,0.000120,0.000000e+00,15.0,71.0,0.0,3.8,NaN,NaN


In [44]:
data1.isnull().sum()

Tair             0
Rhair            0
CO2air           0
co2_dos          0
Tot_PAR          0
T_out            0
RH_out           0
I_glob           0
Windsp           0
Stem_elong    2292
Stem_thick    2292
dtype: int64

In [45]:
#생육 초기에는 생장값이 측정되지 않은 것이므로 0으로 채우는게 맞을듯
data1['Stem_elong'] = data1['Stem_elong'].fillna(0)
data1['Stem_thick'] = data1['Stem_thick'].fillna(0)

/tmp/ipykernel_39869/4045579386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Stem_elong'] = data1['Stem_elong'].fillna(0)
/tmp/ipykernel_39869/4045579386.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Stem_thick'] = data1['Stem_thick'].fillna(0)


In [46]:
#data2 = data.fillna(method='backfill')
#생육 초기에는 생장값이 측정되지 않은 것이므로 0으로 채우는게 맞을듯
data2['Stem_elong'] = data2['Stem_elong'].fillna(method='ffill')
data2['Stem_thick'] = data2['Stem_thick'].fillna(method='ffill')

/tmp/ipykernel_39869/805227564.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Stem_elong'] = data2['Stem_elong'].fillna(method='ffill')
/tmp/ipykernel_39869/805227564.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Stem_thick'] = data2['Stem_thick'].fillna(method='ffill')


In [47]:
data1.isnull().sum()

Tair          0
Rhair         0
CO2air        0
co2_dos       0
Tot_PAR       0
T_out         0
RH_out        0
I_glob        0
Windsp        0
Stem_elong    0
Stem_thick    0
dtype: int64

In [48]:
data2.isnull().sum()

Tair          0
Rhair         0
CO2air        0
co2_dos       0
Tot_PAR       0
T_out         0
RH_out        0
I_glob        0
Windsp        0
Stem_elong    1
Stem_thick    1
dtype: int64

In [49]:
data2['Stem_elong'] = data2['Stem_elong'].fillna(method='backfill')
data2['Stem_thick'] = data2['Stem_thick'].fillna(method='backfill')

/tmp/ipykernel_39869/3362497364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Stem_elong'] = data2['Stem_elong'].fillna(method='backfill')
/tmp/ipykernel_39869/3362497364.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Stem_thick'] = data2['Stem_thick'].fillna(method='backfill')


In [50]:
data2.isnull().sum()

Tair          0
Rhair         0
CO2air        0
co2_dos       0
Tot_PAR       0
T_out         0
RH_out        0
I_glob        0
Windsp        0
Stem_elong    0
Stem_thick    0
dtype: int64

In [51]:
data3 = pd.concat([data1, data2], axis=0)
data3.head()

,Tair,Rhair,CO2air,co2_dos,Tot_PAR,T_out,RH_out,I_glob,Windsp,Stem_elong,Stem_thick
time,,,,,,,,,,,
2019-12-16 00:00:00,21.3,60.7,468.000000,0.001405,0.0,6.9,81.0,0.0,4.7,0.0,0.0
2019-12-16 00:05:00,21.5,62.4,508.999999,0.001405,0.0,6.9,81.0,0.0,4.7,0.0,0.0
2019-12-16 00:10:00,22.0,61.4,498.000001,0.001311,0.0,6.9,80.0,0.0,4.7,0.0,0.0
2019-12-16 00:15:00,21.3,64.8,491.000001,0.000017,0.0,6.9,80.0,0.0,4.7,0.0,0.0
2019-12-16 00:20:00,21.6,64.6,514.999999,0.000019,0.0,6.7,81.0,0.0,4.7,0.0,0.0


In [52]:
data3.tail()

,Tair,Rhair,CO2air,co2_dos,Tot_PAR,T_out,RH_out,I_glob,Windsp,Stem_elong,Stem_thick
time,,,,,,,,,,,
2020-05-29 23:40:00,18.3,77.8,493.0,0.000122,4.999999e-01,15.1,71.0,0.0,4.3,26.8,8.8
2020-05-29 23:45:00,18.3,78.5,493.0,0.000121,5.000000e-01,15.0,71.0,0.0,4.3,26.8,8.8
2020-05-29 23:50:00,18.3,79.1,485.0,0.000121,1.680000e-08,15.0,71.0,0.0,3.8,26.8,8.8
2020-05-29 23:55:00,18.0,79.9,502.0,0.000120,0.000000e+00,15.0,71.0,0.0,3.8,26.8,8.8
2020-05-30 00:00:00,18.0,79.9,502.0,0.000120,0.000000e+00,15.0,71.0,0.0,3.8,26.8,8.8


In [53]:
data3.isnull().sum()

Tair          0
Rhair         0
CO2air        0
co2_dos       0
Tot_PAR       0
T_out         0
RH_out        0
I_glob        0
Windsp        0
Stem_elong    0
Stem_thick    0
dtype: int64

In [54]:
data3 = data3.resample(rule='H').mean()

In [55]:
data3.head()

,Tair,Rhair,CO2air,co2_dos,Tot_PAR,T_out,RH_out,I_glob,Windsp,Stem_elong,Stem_thick
time,,,,,,,,,,,
2019-12-16 00:00:00,21.450000,62.458333,495.416667,0.000360,0.0,6.791667,81.333333,0.0,4.116667,0.0,0.0
2019-12-16 01:00:00,21.066667,60.208333,494.916667,0.000020,0.0,6.300000,84.083333,0.0,2.908333,0.0,0.0
2019-12-16 02:00:00,21.008333,61.508333,494.166667,0.000017,0.0,6.141667,85.500000,0.0,3.200000,0.0,0.0
2019-12-16 03:00:00,21.025000,61.450000,497.250000,0.000014,0.0,6.083333,88.166667,0.0,3.325000,0.0,0.0
2019-12-16 04:00:00,20.891667,61.800000,500.416667,0.000013,0.0,6.025000,90.083333,0.0,3.616667,0.0,0.0


## Pycaret
- target = 'Stem_elong'

In [67]:
from pycaret.regression import *

In [69]:
sup = setup(data3, target = 'Stem_elong', train_size = 0.8)

,Description,Value
0,session_id,2919
1,Target,Stem_elong
2,Original Data,"(3985, 11)"
3,Missing Values,False
4,Numeric Features,10
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(3188, 10)"


In [70]:
# 모델 비교

comp = compare_models(sort = 'RMSE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0487,0.0386,0.1896,0.9995,0.0055,0.0015,0.0220
rf,Random Forest Regressor,0.0420,0.0883,0.2829,0.9988,0.0115,0.0016,0.0510
dt,Decision Tree Regressor,0.0247,0.1490,0.3236,0.9981,0.0131,0.0010,0.0030
catboost,CatBoost Regressor,0.1637,0.1155,0.3359,0.9985,0.0582,0.0055,1.5430
lightgbm,Light Gradient Boosting Machine,0.1064,0.1430,0.3458,0.9982,0.0371,0.0071,0.0110
xgboost,Extreme Gradient Boosting,0.1437,0.1375,0.3625,0.9982,0.0178,0.0048,0.0380
gbr,Gradient Boosting Regressor,0.3756,0.4872,0.6955,0.9937,0.0358,0.0121,0.0500
ada,AdaBoost Regressor,0.8648,1.9263,1.3854,0.9753,0.0412,0.0267,0.0160
lar,Least Angle Regression,1.9794,7.2841,2.6959,0.9076,0.2557,0.0747,0.0020
lr,Linear Regression,1.9794,7.2841,2.6959,0.9076,0.2557,0.0747,0.1750


In [72]:
# 모델 생성

top1 = create_model('et', cross_validation=False)
top2= create_model('rf', cross_validation=False)
top3 = create_model('dt', cross_validation=False)
top4 = create_model('catboost', cross_validation=False)
top5 = create_model('lightgbm', cross_validation=False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0913,0.0751,0.274,0.9989,0.0185,0.0029


In [73]:
# 파라미터 튜닝

tuned_et = tune_model(top1, optimize = 'RMSE', n_iter = 100)
tuned_rf = tune_model(top2, optimize = 'RMSE', n_iter = 100)
tuned_dt = tune_model(top3, optimize = 'RMSE', n_iter = 100)
tuned_catboost = tune_model(top4, optimize = 'RMSE', n_iter = 100)
tuned_lightgbm = tune_model(top5, optimize = 'RMSE', n_iter = 100)

# 27m19s

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.1500,0.0938,0.3063,0.9989,0.0275,0.0045
1,0.1463,0.1383,0.3719,0.9984,0.0296,0.0046
2,0.1462,0.1918,0.4379,0.9970,0.0199,0.0045
3,0.1521,0.1321,0.3635,0.9987,0.0326,0.0047
4,0.1278,0.1180,0.3435,0.9987,0.0273,0.0041
5,0.1303,0.1166,0.3414,0.9986,0.0270,0.0040
6,0.1660,0.1454,0.3814,0.9982,0.0417,0.0050
7,0.1338,0.0874,0.2957,0.9986,0.0259,0.0040
8,0.1655,0.1324,0.3639,0.9981,0.0219,0.0051


In [74]:
# 모델 블렌딩

blender_specific = blend_models(estimator_list = [tuned_et, tuned_rf, tuned_dt, tuned_catboost, tuned_lightgbm],optimize = 'RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0774,0.0359,0.1895,0.9996,0.0092,0.0023
1,0.0855,0.0940,0.3066,0.9989,0.0127,0.0028
2,0.0970,0.2053,0.4531,0.9967,0.0132,0.0029
3,0.0811,0.0433,0.2082,0.9996,0.0088,0.0025
4,0.0616,0.0375,0.1936,0.9996,0.0121,0.0019
5,0.0674,0.0408,0.2020,0.9995,0.0087,0.0020
6,0.1113,0.2363,0.4861,0.9971,0.0163,0.0033
7,0.0638,0.0325,0.1804,0.9995,0.0082,0.0019
8,0.0995,0.1086,0.3296,0.9985,0.0108,0.0031


In [75]:
# train and prediction

pycaret_model = finalize_model(blender_specific)
pred = predict_model(pycaret_model, data=data3)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.0527,0.0252,0.1587,0.9997,0.0129,0.0021
